In [ ]:
from __future__ import annotations

import os 

from torch.utils.data import random_split

from mmpfn.datasets.pad_ufes_20 import PADUFES20Dataset

import os 
import torch 
import numpy as np 
import pandas as pd

from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, root_mean_squared_error

from mmpfn.models.tabpfn_v2 import TabPFNClassifier
from mmpfn.models.dino_v2.models.vision_transformer import vit_base
from mmpfn.models.tabpfn_v2.constants import ModelInterfaceConfig
from mmpfn.models.tabpfn_v2.preprocessing import PreprocessorConfig
from mmpfn.scripts_finetune.finetune_tabpfn_main import fine_tune_tabpfn

/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [ ]:
# data_path = os.path.join(os.getenv('HOME'), "workspace/works/tabular_image/MultiModalPFN/mmpfn/data/pad_ufes_20")
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/pad_ufes_20")
dataset = PADUFES20Dataset(data_path)

In [ ]:
accuracy_scores = []
for seed in range(5):
    torch.manual_seed(seed)
    # np.random.seed(seed)
    # print(f"Finetuning with seed: {seed}")
    
    train_len = int(len(dataset) * 0.8)
    test_len = len(dataset) - train_len

    train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

    X_train = train_dataset.dataset.x[train_dataset.indices]
    y_train = train_dataset.dataset.y[train_dataset.indices]
    X_test = test_dataset.dataset.x[test_dataset.indices]
    y_test = test_dataset.dataset.y[test_dataset.indices]

    for i in range(X_train.shape[1]):
        col = X_train[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1
    for i in range(X_test.shape[1]):
        col = X_test[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1

    torch.cuda.empty_cache()

    save_path_to_fine_tuned_model = "./finetuned_tabpfn_pad_ufes_20.ckpt"
    
    fine_tune_tabpfn(
        # path_to_base_model="auto",
        save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
        # Finetuning HPs
        time_limit=60,
        finetuning_config={"learning_rate": 0.00001, "batch_size": 1, "max_steps": 100},
        validation_metric="log_loss",
        # Input Data
        X_train=pd.DataFrame(X_train),
        y_train=pd.Series(y_train),
        categorical_features_index=None,
        device="cuda",  # use "cpu" if you don't have a GPU
        task_type="multiclass",
        # Optional
        show_training_curve=False,  # Shows a final report after finetuning.
        logger_level=0,  # Shows all logs, higher values shows less
    )

    # disables preprocessing at inference time to match fine-tuning
    no_preprocessing_inference_config = ModelInterfaceConfig(
        FINGERPRINT_FEATURE=False,
        PREPROCESS_TRANSFORMS=[PreprocessorConfig(name='none')]
    )

    # Evaluate on Test Data
    model_finetuned = TabPFNClassifier(
        model_path=save_path_to_fine_tuned_model,
        inference_config=no_preprocessing_inference_config,
        ignore_pretraining_limits=True,
    )

    clf_finetuned = model_finetuned.fit(X_train, y_train)
    acc_score = accuracy_score(y_test, clf_finetuned.predict(X_test))
    print("accuracy_score (Finetuned):", acc_score)
    accuracy_scores.append(acc_score)

Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-08-20 21:41:01,672] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   7%|▋         | 7/100 [00:02<00:34,  2.70it/s, Best Val. Loss=0.562, Best Val. Score=-0.562, Training Loss=0.437, Val. Loss=0.569, Patience=45, Utilization=0, Grad Norm=5.93][2025-08-20 21:41:03,849] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [00:36,  2.71it/s, Best Val. Loss=0.527, Best Val. Score=-0.527, Training Loss=0.457, Val. Loss=0.538, Patience=-48, Utilization=0, Grad Norm=4.82]                         
[2025-08-20 21:41:38,105] INFO - Initial Validation Loss: 0.565506809631864 Best Validation Loss: 0.5268581387941693 Total Steps: 101 Best Step: 45 Total Time Spent: 38.10961365699768


accuracy_score (Finetuned): 0.8434782608695652


Fine-tuning Steps:   6%|▌         | 6/100 [00:02<00:37,  2.53it/s, Best Val. Loss=0.557, Best Val. Score=-0.557, Training Loss=0.545, Val. Loss=0.557, Patience=45, Utilization=0, Grad Norm=3.87][2025-08-20 21:41:41,212] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  28%|██▊       | 28/100 [00:10<00:26,  2.72it/s, Best Val. Loss=0.533, Best Val. Score=-0.533, Training Loss=0.509, Val. Loss=0.533, Patience=24, Utilization=0, Grad Norm=6.2] [2025-08-20 21:41:49,756] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [00:36,  2.73it/s, Best Val. Loss=0.533, Best Val. Score=-0.533, Training Loss=0.476, Val. Loss=0.536, Patience=-48, Utilization=0, Grad Norm=5.42]                         
[2025-08-20 21:42:15,538] INFO - Initial Validation Loss: 0.591181935659142 Best Validation Loss: 0.5328848436445162 Total Steps: 101 Best Step: 65 Total Time Spent: 36.82858920097351


accuracy_score (Finetuned): 0.8217391304347826


Fine-tuning Steps:   3%|▎         | 3/100 [00:00<00:45,  2.15it/s, Best Val. Loss=0.619, Best Val. Score=-0.619, Training Loss=0.465, Val. Loss=0.619, Patience=48, Utilization=0, Grad Norm=4.09][2025-08-20 21:42:17,525] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  14%|█▍        | 14/100 [00:05<00:34,  2.50it/s, Best Val. Loss=0.579, Best Val. Score=-0.579, Training Loss=0.543, Val. Loss=0.579, Patience=38, Utilization=0, Grad Norm=6.03][2025-08-20 21:42:21,852] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [00:37,  2.66it/s, Best Val. Loss=0.555, Best Val. Score=-0.555, Training Loss=0.376, Val. Loss=0.566, Patience=-48, Utilization=0, Grad Norm=4.77]                         
[2025-08-20 21:42:53,933] INFO - Initial Validation Loss: 0.6289394323962494 Best Validation Loss: 0.554768468077869 Total Steps: 101 Best Step: 60 Total Time Spent: 37.871843099594116


...

accuracy_score (Finetuned): 0.8369565217391305


Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-08-20 21:46:00,938] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  19%|█▉        | 19/100 [00:06<00:31,  2.57it/s, Best Val. Loss=0.53, Best Val. Score=-0.53, Training Loss=0.596, Val. Loss=0.53, Patience=33, Utilization=0, Grad Norm=6.08]   [2025-08-20 21:46:07,866] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [00:36,  2.74it/s, Best Val. Loss=0.521, Best Val. Score=-0.521, Training Loss=0.397, Val. Loss=0.525, Patience=-48, Utilization=0, Grad Norm=6.07]                         
[2025-08-20 21:46:37,122] INFO - Initial Validation Loss: 0.5653504830895202 Best Validation Loss: 0.5206775840861005 Total Steps: 101 Best Step: 50 Total Time Spent: 36.70320010185242


accuracy_score (Finetuned): 0.808695652173913


Fine-tuning Steps:   5%|▌         | 5/100 [00:01<00:39,  2.42it/s, Best Val. Loss=0.524, Best Val. Score=-0.524, Training Loss=0.477, Val. Loss=0.524, Patience=46, Utilization=0, Grad Norm=2.73][2025-08-20 21:46:39,877] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  30%|███       | 30/100 [00:11<00:27,  2.53it/s, Best Val. Loss=0.475, Best Val. Score=-0.475, Training Loss=0.506, Val. Loss=0.475, Patience=22, Utilization=0, Grad Norm=5.86][2025-08-20 21:46:49,648] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [00:37,  2.69it/s, Best Val. Loss=0.467, Best Val. Score=-0.467, Training Loss=0.562, Val. Loss=0.475, Patience=-48, Utilization=0, Grad Norm=6.4]                          
[2025-08-20 21:47:15,101] INFO - Initial Validation Loss: 0.5374249483579934 Best Validation Loss: 0.46681897809388295 Total Steps: 101 Best Step: 40 Total Time Spent: 37.45864796638489


accuracy_score (Finetuned): 0.85


In [4]:
# get mean and std of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)
print("Std Accuracy:", std_accuracy)

Mean Accuracy: 0.8304347826086957
Std Accuracy: 0.01816225962974384
